# Logistic Regression with Grid Search (scikit-learn)

<a href="https://colab.research.google.com/github/VertaAI/modeldb-client/blob/master/workflows/demos/sklearn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# restart your notebook if prompted on Colab
try:
    import verta
except ModuleNotFoundError:
    !pip install verta

This example features:
- **scikit-learn**'s `LinearRegression` model
- **scikit-learn**'s `GridSearchCV` utility for performing grid search and cross-validation
- **verta**'s Python client logging the grid search results
- **verta**'s Python client retrieving the best run from the grid search to calculate full training accuracy
- predictions against a deployed model

In [3]:
HOST = "dev.verta.ai"

PROJECT_NAME = "Census Income Classification - Local dir test"
EXPERIMENT_NAME = "Logistic Regression"

In [4]:
import os
os.environ['VERTA_EMAIL'] = 'manasi@verta.ai'
os.environ['VERTA_DEV_KEY'] = 'b1695b7c-2c1a-4076-8a9b-06e105f8b738'

## Imports

In [5]:
import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.filterwarnings("ignore", category=ConvergenceWarning)

import itertools
from multiprocessing import Pool
import os
import time

import six

import numpy as np
import pandas as pd

import sklearn
from sklearn import model_selection
from sklearn import linear_model
from sklearn import metrics

In [6]:
try:
    import wget
except ModuleNotFoundError:
    !pip install wget  # you may need pip3
    import wget

---

# Log Workflow

## Instantiate Client

In [7]:
from verta import Client
from verta.utils import ModelAPI

client = Client(HOST)
proj = client.set_project(PROJECT_NAME)
expt = client.set_experiment(EXPERIMENT_NAME)

set email from environment
set developer key from environment
connection successfully established
Git repository successfully located at /Users/mvartak/Projects/modeldb-client4/
created new Project: Census Income Classification - Local dir test
created new Experiment: Logistic Regression


## Prepare Data

In [8]:
dataset = client.create_local_dataset("census" + str(time.time()))
version = client.create_local_dataset_version(dataset, path="census-train.csv")

created new Dataset: census1563387083.4851031
created new DatasetVersion: 0


In [12]:
DATASET_PATH = "./"

train_data_filename = DATASET_PATH + "census-train.csv"
test_data_filename = DATASET_PATH + "census-test.csv"

In [13]:
df_train = pd.read_csv(train_data_filename)
X_train = df_train.iloc[:,:-1]
y_train = df_train.iloc[:, -1]

df_train.head()

,age,capital-gain,capital-loss,hours-per-week,workclass_local-gov,workclass_private,workclass_self-emp-inc,workclass_self-emp-not-inc,workclass_state-gov,workclass_without-pay,...,occupation_handlers-cleaners,occupation_machine-op-inspct,occupation_other-service,occupation_priv-house-serv,occupation_prof-specialty,occupation_protective-serv,occupation_sales,occupation_tech-support,occupation_transport-moving,>50k
0,44,0,0,40,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,21,0,0,40,0,1,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,53,7298,0,60,0,1,0,0,0,0,...,0,0,0,0,0,0,1,0,0,1
3,49,0,0,40,0,1,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
4,53,0,1485,40,0,1,0,0,0,0,...,0,0,0,0,0,0,1,0,0,1


## Prepare Hyperparameters

In [14]:
hyperparam_candidates = {
    'C': [1e-6, 1e-4],
    'solver': ['lbfgs'],
    'max_iter': [15, 28],
}
hyperparam_sets = [dict(zip(hyperparam_candidates.keys(), values))
                   for values
                   in itertools.product(*hyperparam_candidates.values())]

## Run Validation

In [15]:
def run_experiment(hyperparams):
    # create object to track experiment run
    run = client.set_experiment_run()
    
    # create validation split
    (X_val_train, X_val_test,
     y_val_train, y_val_test) = model_selection.train_test_split(X_train, y_train,
                                                                 test_size=0.2,
                                                                 shuffle=True)

    # log hyperparameters
    run.log_hyperparameters(hyperparams)
    print(hyperparams, end=' ')
    
    # create and train model
    model = linear_model.LogisticRegression(**hyperparams)
    model.fit(X_train, y_train)
    
    # calculate and log validation accuracy
    val_acc = model.score(X_val_test, y_val_test)
    run.log_metric("val_acc", val_acc)
    print("Validation accuracy: {:.4f}".format(val_acc))
    
    # create deployment artifacts
    model_api = ModelAPI(X_train, y_train)
    requirements = six.StringIO("scikit-learn=={}".format(sklearn.__version__))
    
    # save and log model
    run.log_model_for_deployment(model, model_api, requirements, X_train, y_train)
    run.log_dataset_version("train", version)
    
for hyperparams in hyperparam_sets:
    run_experiment(hyperparams)

# with Pool() as pool:
#     pool.map(run_experiment, hyperparam_sets)

created new ExperimentRun: Run 174051563387315044996
{'C': 1e-06, 'solver': 'lbfgs', 'max_iter': 15} Validation accuracy: 0.7879
created new ExperimentRun: Run 174051563387331852971
{'C': 1e-06, 'solver': 'lbfgs', 'max_iter': 28} Validation accuracy: 0.7949
created new ExperimentRun: Run 174051563387344901113
{'C': 0.0001, 'solver': 'lbfgs', 'max_iter': 15} Validation accuracy: 0.7822
created new ExperimentRun: Run 174051563387356902245
{'C': 0.0001, 'solver': 'lbfgs', 'max_iter': 28} Validation accuracy: 0.7840


---

# Revisit Workflow

## Retrieve Best Run

In [ ]:
best_run = expt.expt_runs.sort("metrics.val_acc", descending=True)[0]
print("Validation Accuracy: {:.4f}".format(best_run.get_metric("val_acc")))

best_hyperparams = best_run.get_hyperparameters()
print("Hyperparameters: {}".format(best_hyperparams))

## Train on Full Dataset

In [ ]:
model = linear_model.LogisticRegression(**best_hyperparams, multi_class='auto')
model.fit(X_train, y_train)

## Calculate Accuracy on Full Training Set

In [ ]:
train_acc = model.score(X_train, y_train)
print("Training accuracy: {:.4f}".format(train_acc))

---

# Make Live Predictions

In [ ]:
model_id = 'YOUR_MODEL_ID' 

## Prepare Data

In [ ]:
df_test = pd.read_csv(test_data_filename)
X_test = df_test.iloc[:,:-1]

## Load Deployed Model

In [ ]:
from verta._demo_utils import DeployedModel

deployed_model = DeployedModel(HOST, model_id)

## Query Deployed Model

In [ ]:
for x in itertools.cycle(X_test.values.tolist()):
    print(deployed_model.predict([x]))
    time.sleep(.5)

---